# Task 3-2

Try various implementations of CNNs that are present in lecture materials using MNIST or Fashion MNIST (or any other you wish) dataset. You may try other examples you find in the web as well. Play with training hyper-parameters and network architecture, with dropouts, batch normalization and data generation. Build the table where compare performance (loss, accuracy) of the combinations you made.   

In [32]:

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.python.keras import utils
from tensorflow.python.keras.optimizers import adam_v2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [20]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [21]:
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

In [22]:
X_train.shape

(60000, 28, 28)

In [23]:
num_classes = 10
img_rows, img_cols = 28, 28

In [24]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [25]:
if tf.keras.backend.image_data_format() == 'channels_first':
    x_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    x_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    x_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [26]:
y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [27]:
res = pd.DataFrame(columns=['name', 'accuracy', 'loss', 'layers', 'dense_layers', 'learning_rate', 'batch_size', 'epochs'])

In [35]:
def create_model(train_params):
    model = Sequential()
    
    for i, layer in enumerate(train_params['layers']):
        if i == 0:
            model.add(Conv2D(layer['filters'], layer['kernel_size'], activation=layer['activation'], input_shape=input_shape))
        else:
            model.add(Conv2D(layer['filters'], layer['kernel_size'], activation=layer['activation']))
        if layer['batch_norm']:
            model.add(BatchNormalization())
        if layer['pool_size']:
            model.add(MaxPooling2D(pool_size=layer['pool_size']))
        if layer['dropout']:
            model.add(Dropout(layer['dropout']))
        
    model.add(Flatten())
    
    for i, dense_layer in enumerate(train_params['dense_layers']):
        model.add(Dense(dense_layer['units'], activation=dense_layer['activation']))
        if dense_layer['batch_norm']:
            model.add(BatchNormalization())
        if dense_layer['dropout']:
            model.add(Dropout(dense_layer['dropout']))
    
    model.add(Dense(num_classes, activation='softmax'))
    optimizer = adam_v2.Adam(learning_rate=train_params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [34]:
train_params = [
    {
        'name': 'simplest',
        'layers': [
            {'filters': 64, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': False, 'pool_size': (2, 2), 'dropout': 0.25},
        ],
        'dense_layers': [
            {'units': 128, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.5},
        ],
        'learning_rate': 0.01,
        'batch_size': 64,
        'epochs': 5
    },
    {
        'name': 'deep',
        'layers': [
            {'filters': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': (2, 2), 'dropout': 0.2},
            {'filters': 64, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': (2, 2), 'dropout': 0.2},
            {'filters': 128, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': None, 'dropout': 0.3}
        ],
        'dense_layers': [
            {'units': 64, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.2},
            {'units': 32, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.2}
        ],
        'learning_rate': 0.01,
        'batch_size': 64,
        'epochs': 5
    },
    {
        'name': 'deepest',
        'layers': [
            {'filters': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': (2, 2), 'dropout': 0.2},
            {'filters': 64, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': (2, 2), 'dropout': 0.2},
            {'filters': 128, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': None, 'dropout': 0.5},
            {'filters': 256, 'kernel_size': (3, 3), 'activation': 'relu', 'batch_norm': True, 'pool_size': None, 'dropout': 0.4}
        ],
        'dense_layers': [
            {'units': 128, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.2},
            {'units': 64, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.2},
            {'units': 32, 'activation': 'relu', 'batch_norm': True, 'dropout': 0.2}
        ],
        'learning_rate': 0.001,
        'batch_size': 64,
        'epochs': 5
    }
]

In [36]:
results = []

for params in train_params:
    if not os.path.isdir(f"./models/{params['name']}"):
        model = create_model(params)
        history = model.fit(x_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], validation_data=(x_test, y_test), callbacks=[early_stop])
        model.save(f"./models/{params['name']}")
    else:
        model = load_model(f"./models/{params['name']}")
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    results.append({
        'params': params,
        'accuracy': accuracy,
        'loss': loss
    })

for i, r in enumerate(results):
    params = r['params']
    accuracy, loss = r['accuracy'], r['loss']
    layers = ', '.join([f"{layer['filters']}{layer['kernel_size'][0]}x{layer['kernel_size'][1]}{layer['activation']}{'bn' if layer['batch_norm'] else ''}{'dr' if layer['dropout'] > 0 else ''}{layer['dropout']}" for layer in params['layers']])
    dense_layers = ', '.join([f"{layer['units']}{'bn' if layer['batch_norm'] else ''}{'dr' if layer['dropout'] > 0 else ''}{layer['dropout']}" for layer in params['dense_layers']])
    res = res.append({
        'name': params['name'],
        'layers': layers,
        'dense_layers': dense_layers,
        'learning_rate': params['learning_rate'],
        'batch_size': params['batch_size'],
        'epochs': params['epochs'],
        'loss': loss,
        'accuracy': accuracy,
    }, ignore_index=True)

print(res)

Epoch 1/5
938/938 [==============================] - 42s 44ms/step - loss: 0.4777 - accuracy: 0.8324 - val_loss: 0.3895 - val_accuracy: 0.8622
Epoch 2/5
938/938 [==============================] - 39s 42ms/step - loss: 0.3673 - accuracy: 0.8697 - val_loss: 0.3401 - val_accuracy: 0.8733
Epoch 3/5
938/938 [==============================] - 35s 37ms/step - loss: 0.3379 - accuracy: 0.8789 - val_loss: 0.3210 - val_accuracy: 0.8855
Epoch 4/5
938/938 [==============================] - 34s 37ms/step - loss: 0.3180 - accuracy: 0.8855 - val_loss: 0.2839 - val_accuracy: 0.8949
Epoch 5/5
938/938 [==============================] - 33s 36ms/step - loss: 0.3023 - accuracy: 0.8915 - val_loss: 0.2899 - val_accuracy: 0.8958
INFO:tensorflow:Assets written to: ./models/simplest\assets


INFO:tensorflow:Assets written to: ./models/simplest\assets


Epoch 1/5
938/938 [==============================] - 40s 42ms/step - loss: 0.5366 - accuracy: 0.8048 - val_loss: 0.4191 - val_accuracy: 0.8427
Epoch 2/5
938/938 [==============================] - 39s 42ms/step - loss: 0.3953 - accuracy: 0.8606 - val_loss: 0.3347 - val_accuracy: 0.8785
Epoch 3/5
938/938 [==============================] - 39s 41ms/step - loss: 0.3548 - accuracy: 0.8745 - val_loss: 0.3091 - val_accuracy: 0.8912
Epoch 4/5
938/938 [==============================] - 39s 42ms/step - loss: 0.3365 - accuracy: 0.8810 - val_loss: 0.3209 - val_accuracy: 0.8809
Epoch 5/5
938/938 [==============================] - 39s 41ms/step - loss: 0.3175 - accuracy: 0.8885 - val_loss: 0.4708 - val_accuracy: 0.8361
INFO:tensorflow:Assets written to: ./models/deep\assets


INFO:tensorflow:Assets written to: ./models/deep\assets


Epoch 1/5
938/938 [==============================] - 43s 45ms/step - loss: 0.7756 - accuracy: 0.7290 - val_loss: 0.4711 - val_accuracy: 0.8256
Epoch 2/5
938/938 [==============================] - 58s 61ms/step - loss: 0.4819 - accuracy: 0.8338 - val_loss: 0.3952 - val_accuracy: 0.8633
Epoch 3/5
938/938 [==============================] - 46s 49ms/step - loss: 0.4183 - accuracy: 0.8554 - val_loss: 0.3337 - val_accuracy: 0.8816
Epoch 4/5
938/938 [==============================] - 45s 48ms/step - loss: 0.3836 - accuracy: 0.8671 - val_loss: 0.3725 - val_accuracy: 0.8649
Epoch 5/5
938/938 [==============================] - 45s 48ms/step - loss: 0.3620 - accuracy: 0.8751 - val_loss: 0.2948 - val_accuracy: 0.8941
INFO:tensorflow:Assets written to: ./models/deepest\assets


INFO:tensorflow:Assets written to: ./models/deepest\assets


       name  accuracy      loss  \
0  simplest    0.8958  0.289879   
1      deep    0.8361  0.470821   
2   deepest    0.8941  0.294753   

                                              layers  \
0                                    643x3reludr0.25   
1  323x3relubndr0.2, 643x3relubndr0.2, 1283x3relu...   
2  323x3relubndr0.2, 643x3relubndr0.2, 1283x3relu...   

                       dense_layers  learning_rate batch_size epochs  
0                        128bndr0.5          0.010         64      5  
1              64bndr0.2, 32bndr0.2          0.010         64      5  
2  128bndr0.2, 64bndr0.2, 32bndr0.2          0.001         64      5  


C:\Users\askador\AppData\Local\Temp\ipykernel_7016\1801560067.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({
C:\Users\askador\AppData\Local\Temp\ipykernel_7016\1801560067.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({
C:\Users\askador\AppData\Local\Temp\ipykernel_7016\1801560067.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({


In [39]:
res.sort_values('accuracy', ascending=False)

,name,accuracy,loss,layers,dense_layers,learning_rate,batch_size,epochs
0,simplest,0.8958,0.289879,643x3reludr0.25,128bndr0.5,0.010,64,5
2,deepest,0.8941,0.294753,"323x3relubndr0.2, 643x3relubndr0.2, 1283x3relu...","128bndr0.2, 64bndr0.2, 32bndr0.2",0.001,64,5
1,deep,0.8361,0.470821,"323x3relubndr0.2, 643x3relubndr0.2, 1283x3relu...","64bndr0.2, 32bndr0.2",0.010,64,5
